## Atari Breakout

Action Space: Discrete(4)
Observation Space: Box(0, 255,(210, 160, 3), unit8) --> an image

import: gym.make("Breakout-v0")
Algorithm: A2C
Policy:"CnnPolicy"

# 1. Import dependencies

In [8]:
pip install stable-baselines3[extra]

SyntaxError: invalid syntax (4048873974.py, line 1)

In [3]:
pip install ale-py

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install pygame

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install pyglet

Note: you may need to restart the kernel to use updated packages.


In [1]:
import os  # operating systme library to save and load model
import gym  # openAI gym simulated environments- building the environment and working with ore-existing
import random
from stable_baselines3 import A2C # one of the algorithms of stable baselines
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecFrameStack  # vectorizing the environements for ML for using 4 environments
from stable_baselines3.common.evaluation import evaluate_policy  # makes it easier to test out if the model is performing
from stable_baselines3.common.env_util import make_atari_env  # helps us works with atari environment

# 2. Load Environment

In [2]:
from ale_py import ALEInterface
from ale_py.roms import Breakout

ale = ALEInterface()
ale.loadROM(Breakout)

A.L.E: Arcade Learning Environment (version 0.7.5+db37282)
[Powered by Stella]
Game console created:
  ROM file:  /Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/AutoROM/roms/breakout.bin
  Cart Name: Breakout - Breakaway IV (1978) (Atari)
  Cart MD5:  f34f08e5eb96e500e851a80be3277a56
  Display Format:  AUTO-DETECT ==> NTSC
  ROM Size:        2048
  Bankswitch Type: AUTO-DETECT ==> 2K

Running ROM file...
Random seed is 1657555204


In [3]:
env = gym.make('ALE/Breakout-v5',
    obs_type='rgb',                   # ram | rgb | grayscale
    frameskip=4,                      # frame skip
    mode=None,                        # game mode, see Machado et al. 2018
    difficulty=None,                  # game difficulty, see Machado et al. 2018
    repeat_action_probability=0.25,   # Sticky action probability
    full_action_space=False,          # Use all actions
    render_mode='human'                  # None | human | rgb_array
)

A.L.E: Arcade Learning Environment (version 0.7.5+db37282)
[Powered by Stella]
2022-07-11 18:00:07.791 Python[3489:121377] ApplePersistenceIgnoreState: Existing state will not be touched. New state will be written to /var/folders/21/_7zvs2610fxgclkhl47pkcm40000gn/T/org.python.python.savedState


In [4]:
# understanding the environment - writing a loop to test that environment
env = gym.make('ALE/Breakout-v5',
    obs_type='rgb',                   # ram | rgb | grayscale
    frameskip=4,                      # frame skip
    mode=None,                        # game mode, see Machado et al. 2018
    difficulty=None,                  # game difficulty, see Machado et al. 2018
    repeat_action_probability=0.25,   # Sticky action probability
    full_action_space=False,          # Use all actions
    render_mode='human'               # None | human | rgb_array
)

episodes = 5  # testing the environment 5 times 
for episode in range(1, episodes+1):
    state = env.reset()  # reseting environment by getting an array of 4 initial set of observations of that perticular environment
    done = False
    score = 0 
    
    while not done:  
        env.render(mode='rgb_array')  # to view the graphical representation of environment (does not work)
        action = env.action_space.sample()  # generating a random action, 0 or 1 - here action_space is Discrete(2) -> {0, 1}
        n_state, reward, done, info = env.step(action)  # passing random action to environment (passing 0 or 1)
        score+=reward  # accumulating rewards
    print('Episode:{} Score:{}'.format(episode, score))

Episode:1 Score:2.0
Episode:2 Score:3.0
Episode:3 Score:1.0
Episode:4 Score:0.0
Episode:5 Score:0.0


In [5]:
env.close()

# Understanding The Environment

In [7]:
# discrete(4) --> 0 NOOP, 1 FIRE, 2 RIGHT, 3 LEFT 
env.action_space

Discrete(4)

In [ ]:
env.unwrapped.get_action_meanings()

In [17]:
env.action_space.sample()

0

In [18]:
# shape:(210, 160, 3)
# The 128 Bytes of RAM of the console--> Box([0 ... 0], [255 ... 255], (128,), uint8)
# A grayscale image --> Box([[0 ... 0] ... [0  ... 0]], [[255 ... 255] ... [255  ... 255]], (250, 160), uint8)
env.observation_space.sample()

array([[[190, 247, 153],
        [216, 193, 232],
        [ 53,  29, 154],
        ...,
        [ 28, 181,  29],
        [116, 213,   7],
        [147, 225,  86]],

       [[182,   9, 255],
        [ 70, 111, 124],
        [ 82,  61, 198],
        ...,
        [235, 189,  90],
        [123, 244,  79],
        [226,  88, 173]],

       [[171, 202, 248],
        [195,   8, 158],
        [ 87,  11, 152],
        ...,
        [132,   4, 228],
        [ 12, 104,  66],
        [162,  68, 155]],

       ...,

       [[119, 223, 235],
        [  3,  70, 168],
        [237,  43, 130],
        ...,
        [139, 162, 232],
        [245, 195, 188],
        [113,  25,  81]],

       [[110, 146,  51],
        [227, 212,  90],
        [122, 217,  23],
        ...,
        [211,  17, 106],
        [122, 177,  18],
        [240, 201,   6]],

       [[147,  42, 232],
        [ 56, 198,  44],
        [203,  21,  78],
        ...,
        [233, 137,  15],
        [165, 119,  17],
        [254,  73,  39]]

In [ ]:
print(env.observation_space.shape)

# 2.2 Vectorizing the Environment and Training


RL Algorithms:
    1. Model-Based - predict future states to generate best possible action
        1. Learn the Model
        2. Given the Model --> AlphaZero
    2. Model-Free ( a lot of developments are happening) - uses current state to make predictions
        1.Policiy Optimization --> PPO (Proximal Policy Optimization ) / A2C / DQN
        2.Q-Learning
using best algorithms based on the action space in Stable Baseline:
    A2C - works on all types of action space (Box, Discrete, MultiDistricrete, MultiBinary, MultiProcessing)
    PPO - works on all types of action space (Box, Discrete, MultiDistricrete, MultiBinary, MultiProcessing)
    DQN - only on Discrete

In [4]:
env = make_atari_env('ALE/Breakout-v5', n_envs=4, seed=0)  # type of environment we want to run (4 environments)
env = VecFrameStack(env, n_stack=4)  # vectorizing the environment for training 4 different environments - stacking those environments together

A.L.E: Arcade Learning Environment (version 0.7.5+db37282)
[Powered by Stella]


In [7]:
env.render()

In [14]:
env.reset()  # not working

# noops = self.unwrapped.np_random.integers(1, self.noop_max + 1) 
# instead of noops = self.unwrapped.np_random.randint(1, self.noop_max + 1)

AttributeError: 'numpy.random._generator.Generator' object has no attribute 'randint'

In [15]:
log_path = os.path.join('Training', 'Logs')  # define a log_path where we can monitor the performance of the model
model = A2C("CnnPolicy", env, verbose=1, tensorboard_log=log_path) # defining the model - defining the policiy (CNN because we are dealing with images)

#model = PPO('MlpPolicy', env, verbose = 1, tensorboard_log=log_path)

Using cpu device
Wrapping the env in a VecTransposeImage.


In [16]:
model.learn(total_timesteps=100000)  # traning the model

AttributeError: 'numpy.random._generator.Generator' object has no attribute 'randint'

# 4. Save and Reload Model

In [2]:
a2c_path = os.path.join('Training', 'Saved Models', 'A2C_300k_model')

In [21]:
a2c_path

'Training/Saved Models/A2C_300k_model'

In [18]:
model.save(a2c_path)

In [20]:
del model

In [22]:
model = A2C.load(a2c_path, env)  # reloading does not work

Wrapping the env in a VecTransposeImage.


In [23]:
model.learn(total_timesteps=1000)

AttributeError: 'numpy.random._generator.Generator' object has no attribute 'randint'

# 4. Test and Evaluation 

In [24]:
env = make_atari_env('Breakout-v0', n_envs=1, seed=0)
env = VecFrameStack(env, n_stack=4)

In [25]:
evaluate_policy(model, env, n_eval_episodes=10, render=True) # average reward (if above 4 then is considered solved, standard deviation

AttributeError: 'numpy.random._generator.Generator' object has no attribute 'randint'

In [26]:
env.close()

# 6. Viewing Logs in Tensorboard (useful for big models)

In [ ]:
pip install tensorboard

In [ ]:
training_log_path = os.path.join(log_path, 'PPO_9')

In [ ]:
!tensorboard --logdir={training_log_path}

# 7. Adding a callback to the training Stage (useful for big models)

In [ ]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold
import os

In [ ]:
save_path = os.path.join('Training', 'Saved Models')
log_path = os.path.join('Training', 'Logs')

In [ ]:
env = gym.make(environment_name)
env = DummyVecEnv([lambda: env])

In [ ]:
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=200, verbose=1)  # stopping the training at 200 score
eval_callback = EvalCallback(env, 
                             callback_on_new_best=stop_callback, 
                             eval_freq=10000,  # on every 10000 steps it checks if it has passed score 200, and if so it saves the best new model
                             best_model_save_path=save_path, 
                             verbose=1)

In [ ]:
model = PPO('MlpPolicy', env, verbose = 1, tensorboard_log=log_path)

In [ ]:
model.learn(total_timesteps=20000, callback=eval_callback)  # 

In [ ]:
model_path = os.path.join('Training', 'Saved Models', 'best_model')
model = PPO.load(model_path, env=env)

In [ ]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

In [ ]:
env.close()

# 8. Changing Policies - using a different network architecture

In [ ]:
net_arch=[dict(pi=[128, 128, 128, 128], vf=[128, 128, 128, 128])]  # new neural architecture defined

In [ ]:
model = PPO('MlpPolicy', env, verbose = 1, policy_kwargs={'net_arch': net_arch})

In [ ]:
model.learn(total_timesteps=20000, callback=eval_callback)

# 9. Using an Alternate Algorithm - DQN instead of PPO

In [ ]:
from stable_baselines3 import DQN

In [ ]:
model = DQN('MlpPolicy', env, verbose = 1, tensorboard_log=log_path)

In [ ]:
model.learn(total_timesteps=20000, callback=eval_callback)

In [ ]:
dqn_path = os.path.join('Training', 'Saved Models', 'DQN_model')

In [ ]:
model.save(dqn_path)

In [ ]:
model = DQN.load(dqn_path, env=env)

In [ ]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

In [ ]:
env.close()